In [2]:
# import sys
# !{sys.executable} -m pip install pip earthengine-api
# !{sys.executable} -m pip install pip geemap

In [3]:
import ee
# ee.Authenticate()

In [4]:
ee.Initialize()

In [5]:
import geemap
import ipyleaflet
import numpy as np
import requests
import os
import pandas as pd
import rasterio
import boto3
import geopandas as gpd
import glob

In [6]:
# define directory
out_dir = os.getcwd()
bucket_name = 'cities-cities4forests' 
aws_s3_dir = 'https://'+bucket_name+'.s3.eu-west-3.amazonaws.com/data'

# Load data

In [7]:
# read forest carbon flux data
# paper: https://www.nature.com/articles/s41558-020-00976-6 
# on GEE: https://code.earthengine.google.com/b21e2ea3103b09710e5ff5004e6d2617

netFlux = ee.ImageCollection('projects/wri-datalab/gfw-data-lake/net-flux-forest-extent-per-ha-v1-2-2-2001-2021/net-flux-global-forest-extent-per-ha-2001-2021')
net_flux = netFlux.reduce(ee.Reducer.mean()); # for use in visualizations
net_fluxw0 = net_flux.unmask(0) # unmask to 0 to enable pixels without tree carbon flux to be included in mean calculation
fluxScale = net_flux.projection().nominalScale()
# Map.addLayer(net_flux,{palette:['green','white','red']})
# Map.addLayer(net_fluxw0,{palette:['green','white','red']},"",0)


In [9]:
# get list of cities
boundary_georef = pd.read_csv(aws_s3_dir+'/boundaries/v_0/boundary_georef.csv')
boundary_georef

,city_name,geo_name,aoi_boundary_name,units_boundary_name,city_boundary_name,country_code,geo_level
0,Salvador,BRA-Salvador,ADM4union,ADM4,BRA-Salvador-ADM4,BRA,ADM4
1,Bukavu,COD-Bukavu,ADM3union,ADM3,COD-Bukavu-ADM3,COD,ADM3
2,Uvira,COD-Uvira,ADM3union,ADM3,COD-Uvira-ADM3,COD,ADM3
3,Brazzaville,COG-Brazzaville,ADM4union,ADM4,COG-Brazzaville-ADM4,COG,ADM4
4,Barranquilla,COL-Barranquilla,ADM4union,ADM4,COL-Barranquilla-ADM4,COL,ADM4
5,Addis_Ababa,ETH-Addis_Ababa,ADM4union,ADM4,ETH-Addis_Ababa-ADM4,ETH,ADM4
6,Dire_Dawa,ETH-Dire_Dawa,ADM3union,ADM3,ETH-Dire_Dawa-ADM3,ETH,ADM3
7,Nairobi,KEN-Nairobi,ADM3union,ADM3,KEN-Nairobi-ADM3,KEN,ADM3
8,Antananarivo,MDG-Antananarivo,ADM4union,ADM4,MDG-Antananarivo-ADM4,MDG,ADM4
9,Mexico_City,MEX-Mexico_City,ADM2union,ADM2,MEX-Mexico_City-ADM2,MEX,ADM2


# Extract data by city

In [12]:
for i in range(0,len(boundary_georef)):
    print(i)
    boundary_id = boundary_georef.loc[i, 'geo_name']+'-'+boundary_georef.loc[i, 'aoi_boundary_name']
    print(boundary_id)
    
    # read boundaries
    boundary_path = aws_s3_dir+'/boundaries/v_0/boundary-'+boundary_id+'.geojson'
    boundary_geo = requests.get(boundary_path).json()
    boundary_geo_ee = geemap.geojson_to_ee(boundary_geo)
    
    # Download ee.Image for TML as GeoTIFF
    geemap.ee_export_image_to_drive(
        net_flux, 
        description = boundary_id + '-WRI-ForestCarbonFluxes',
        folder='data', 
        scale=30, 
        region=boundary_geo_ee.geometry(),
        maxPixels = 5000000000
    )

1
COD-Bukavu-ADM3union
2
COD-Uvira-ADM3union
3
COG-Brazzaville-ADM4union
4
COL-Barranquilla-ADM4union
5
ETH-Addis_Ababa-ADM4union
6
ETH-Dire_Dawa-ADM3union
7
KEN-Nairobi-ADM3union
8
MDG-Antananarivo-ADM4union
9
MEX-Mexico_City-ADM2union
10
MEX-Monterrey-ADM2union
11
RWA-Musanze-ADM5union


# Upload in aws

Since we can't download directly the rasters locally due to their size, the rasters are stored in a google-drive folder and then downloaded locally in order to push them back to s3 bucket.

In [13]:
# connect to s3
aws_credentials = pd.read_csv('/home/jovyan/PlanetaryComputerExamples/aws_credentials.csv')
aws_key = aws_credentials.iloc[0]['Access key ID']
aws_secret = aws_credentials.iloc[0]['Secret access key']

s3 = boto3.resource(
    service_name='s3',
    aws_access_key_id=aws_key,
    aws_secret_access_key=aws_secret
)

In [15]:
for i in range(0, len(boundary_georef)):
    boundary_id = boundary_georef.loc[i, 'geo_name']+'-'+boundary_georef.loc[i, 'aoi_boundary_name']
    geo_name = boundary_georef.loc[i, 'geo_name']
    print("\n boundary_id: "+boundary_id)

    # read local raster
    city_file = 'data/' + boundary_id +'-WRI-ForestCarbonFluxes.tif'
    raster_path = os.path.join(out_dir, city_file)
    
    # upload in s3
    s3.meta.client.upload_file(raster_path, 
                               bucket_name, 
                               'data/tree_cover/wri-forest-carbon-fluxes/v_0/'+ boundary_id + '-WRI-ForestCarbonFluxes-MgCO2eperHA2001-2021.tif',
                               ExtraArgs={'ACL':'public-read'})


 boundary_id: BRA-Salvador-ADM4union

 boundary_id: COD-Bukavu-ADM3union

 boundary_id: COD-Uvira-ADM3union

 boundary_id: COG-Brazzaville-ADM4union

 boundary_id: COL-Barranquilla-ADM4union

 boundary_id: ETH-Addis_Ababa-ADM4union

 boundary_id: ETH-Dire_Dawa-ADM3union

 boundary_id: KEN-Nairobi-ADM3union

 boundary_id: MDG-Antananarivo-ADM4union

 boundary_id: MEX-Mexico_City-ADM2union

 boundary_id: MEX-Monterrey-ADM2union

 boundary_id: RWA-Musanze-ADM5union
